<a href="https://colab.research.google.com/github/Yversjcl/Entregas_Coderhouse_YJCL/blob/main/C_DesafiosC/Desafio_CrossValidation_Yvers_Caruzo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Desafío Crossvalidation y mejoras de modelos de ML

Este desafío viene a complementar la entrega anterior vinculada al Feature selection que hemos venido trabajando, relacionada al análisis de una propuesta de rentabilización para la industria de telco, a partir del posicionamiento de ofertas a clientes del segmento prepago.

El objetivo principal es la creación de un modelo que permita predecir el comportamiento de clientes migrados y el performance de la iniciatriva de cara a la mejora para inv¿crementar la conversión de la misma.

Para ello trabajamos **Random Forest** alcanzando un **Accuracy: 0.7420382165605095** de allí que entremos a validar el rendimiento a partir del crossvalidation y así mirar la consistencia en el desempeño del modelo en general.


1. Importación de librerias

In [11]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
# Establecer ruta de acceso en dr
import os
print(os.getcwd())
os.chdir("/content/gdrive/My Drive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive


In [12]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.compose import ColumnTransformer

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

2. Carga y revisión del dataset

In [3]:
# Lectura de DF
df=pd.read_csv('/content/gdrive/MyDrive/BBDD_MIS_T_COMP.csv', delimiter=';')

In [13]:
df.shape

(3134, 9)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3134 entries, 0 to 3133
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   Tipo_Cliente        3134 non-null   int64
 1   Oferta              3134 non-null   int64
 2   Escucha_Oferta_Mig  3134 non-null   int64
 3   Acepta_Mig          3134 non-null   int64
 4   Valida_Rut          3134 non-null   int64
 5   EvalCredito         3134 non-null   int64
 6   T&C                 3134 non-null   int64
 7   migradoOK           3134 non-null   int64
 8   Envio_RPA           3134 non-null   int64
dtypes: int64(9)
memory usage: 220.5 KB


**Descripcion de variables presentes en el dataset**

1. Tipo_Cliente: Define a tipificación de cliente con y sin identificación.
2. Oferta: Tipo de campaña cargada y ofrecida en prompt al cliente.
3. Escucha_Oferta_Mig: cantidad de clientes que escucharon o no la oferta presentada.
4. Acepta_Mig: cantidad de clientes que aceptan o no la oferta presentada
5. Valida_Rut: indica cantidad de clientes que validaron o no su identidad
6. EvalCredito: Indica cantidad de clientes que pasaron de manera exitosa la evaluación comercial.
7. T&C: Indica cantidad de clientes que aceptaron terminos y condiciones comerciales del plan presentado.
8. migradoOK: cantidad de clientes que cambian su tipo de contrato
9. Envio_RPA: cantidad de clientes que no logran culminar su proceso en el flujo automático y se envian a la asistencia de un ejecutivo.

In [16]:
df

,Tipo_Cliente,Oferta,Escucha_Oferta_Mig,Acepta_Mig,Valida_Rut,EvalCredito,T&C,migradoOK,Envio_RPA
0,1,0,743,69,26,1,0,5,0
1,1,1,316,22,3,0,0,17,0
2,1,0,4,0,0,0,0,7,0
3,1,1,346,22,8,0,0,17,0
4,1,0,10,0,0,0,0,9,0
...,...,...,...,...,...,...,...,...,...
3129,0,0,4219,472,175,18,17,42,17
3130,0,0,2691,231,78,8,6,42,6
3131,0,0,5157,568,269,29,24,45,24
3132,0,0,4483,489,217,26,21,61,21


import numpy as np
from google.colab import autoviz
df_3786112700880471170 = autoviz.get_df('df_3786112700880471170')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_3786112700880471170, *['Tipo_Cliente'], **{})
chart

import numpy as np
from google.colab import autoviz
df_3786112700880471170 = autoviz.get_df('df_3786112700880471170')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_3786112700880471170, *['Oferta'], **{})
chart

import numpy as np
from google.colab import autoviz
df_3786112700880471170 = autoviz.get_df('df_3786112700880471170')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_3786112700880471170, *['Escucha_Oferta_Mig'], **{})
chart

import numpy as np
from google.colab import autoviz
df_3786112700880471170 = autoviz.get_df('df_3786112700880471170')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_3786112700880471170, *['Acepta_Mig'], **{})
chart

import numpy as np
from google.colab import autoviz
df_3786112700880471170 = autoviz.get_df('df_3786112700880471170')

def histogram(df, colname, num_bins=20, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  _, ax = plt.subplots(figsize=figsize)
  plt.hist(df[colname], bins=num_bins, histtype='stepfilled')
  plt.ylabel('count')
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(df_3786112700880471170, *['Tipo_Cliente'], **{})
chart

import numpy as np
from google.colab import autoviz
df_3786112700880471170 = autoviz.get_df('df_3786112700880471170')

def histogram(df, colname, num_bins=20, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  _, ax = plt.subplots(figsize=figsize)
  plt.hist(df[colname], bins=num_bins, histtype='stepfilled')
  plt.ylabel('count')
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(df_3786112700880471170, *['Oferta'], **{})
chart

import numpy as np
from google.colab import autoviz
df_3786112700880471170 = autoviz.get_df('df_3786112700880471170')

def histogram(df, colname, num_bins=20, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  _, ax = plt.subplots(figsize=figsize)
  plt.hist(df[colname], bins=num_bins, histtype='stepfilled')
  plt.ylabel('count')
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(df_3786112700880471170, *['Escucha_Oferta_Mig'], **{})
chart

import numpy as np
from google.colab import autoviz
df_3786112700880471170 = autoviz.get_df('df_3786112700880471170')

def histogram(df, colname, num_bins=20, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  _, ax = plt.subplots(figsize=figsize)
  plt.hist(df[colname], bins=num_bins, histtype='stepfilled')
  plt.ylabel('count')
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(df_3786112700880471170, *['Acepta_Mig'], **{})
chart

import numpy as np
from google.colab import autoviz
df_3786112700880471170 = autoviz.get_df('df_3786112700880471170')

def scatter_plots(df, colname_pairs, scatter_plot_size=2.5, size=8, alpha=.6):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * scatter_plot_size, scatter_plot_size))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    ax.scatter(df[x_colname], df[y_colname], s=size, alpha=alpha)
    plt.xlabel(x_colname)
    plt.ylabel(y_colname)
    ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plots(df_3786112700880471170, *[[['Tipo_Cliente', 'Oferta'], ['Oferta', 'Escucha_Oferta_Mig'], ['Escucha_Oferta_Mig', 'Acepta_Mig'], ['Acepta_Mig', 'Valida_Rut']]], **{})
chart

3. Modelado de datos

In [17]:
# Dividir el conjunto de datos en características (X) y etiquetas (y)
X = df.drop(['migradoOK'], axis=1)
y = df['migradoOK']

In [18]:
# Crear el modelo de Random Forest con los hiperparámetros deseados
random_forest_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

In [19]:
#Encoding y manejo de valores faltantes
# Codificación de variables categóricas
cat_features = ['Tipo_Cliente', 'Oferta']
num_features = ['Escucha_Oferta_Mig', 'Acepta_Mig', 'Valida_Rut', 'EvalCredito', 'T&C']
target = 'migradoOK'

In [20]:
# Codificación de variables categóricas
label_encoder = LabelEncoder()
onehot_encoder = OneHotEncoder(drop='first', sparse=False)
transformers = [('encoder', onehot_encoder, cat_features)]
column_transformer = ColumnTransformer(transformers, remainder='passthrough')

X = column_transformer.fit_transform(df.drop(target, axis=1))
y = df[target]

4. Crossvalidation

In [22]:
# Realizar cross-validation con 10 folds y evaluar el rendimiento usando la precisión
cv_scores = cross_val_score(random_forest_model, X, y, cv=10, scoring='accuracy')

In [23]:
# Obtener el promedio de las métricas de cross-validation
average_cv_accuracy = cv_scores.mean()

print("Random Forest - Cross-Validation Accuracy:", average_cv_accuracy)

Random Forest - Cross-Validation Accuracy: 0.7393541034980973


**Consideraciones proceso de Crossvalidation**

Como se indicó al inicio y viendo el proceso de crossvalidation el modelo presenta un accuracy entorno al 0.74, visto el **Random Forest - Cross-Validation Accuracy: 0.7393541034980973**, lo que muestra una correspondencia entre ambos modelos.

Así mismo recordar la mejora que se realiza al modelo y que se evidencio en los cambnios expresados en las métricas entre las rondas 1 y 2 previas al feature selection, lo que pareciera indicar que para el objetivo de predicción del la variable "migradoOK" nos entrega exito en el 74% de los casos en general y en algunos de los paremetros presenta un rendimiento mayor.

Otro aspecto importante, es el desempeño a partir de las variables sintéticas que presenta un comportamiento mejor en los componentes de no aceptación del plan propuesto a nivel comercial, esto a su vez es insight para la toma decisiones para la intervención y mejora del flujo diseñado de cara a clientes objetivo.

